In [ ]:
import gmsh
import numpy as np

el metodo initialize 'abre' el programa.

In [ ]:
gmsh.initialize()

'armamos' el modelo que queremos modificar:

In [ ]:
gmsh.model.add('test2')

In [ ]:
L = 10
lc = 2*L

Definimos las coordenadas de los puntos

In [ ]:
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc/10) #/5)
p2 = gmsh.model.geo.addPoint(2*L, 0, 0, lc/2) #*2) 
p3 = gmsh.model.geo.addPoint(2*L, L, 0, lc/2) #/2) 
p4 = gmsh.model.geo.addPoint(0, L, 0, lc/10) #/5) 

Luego definimos las lineas

In [ ]:
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

Las curvas nos van a servir para definir los bordes de nuestro modelo.

In [ ]:
C1 = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])

Y las superficies:

In [ ]:
S1 = gmsh.model.geo.addPlaneSurface([C1])

le avisamos que ya estan todos los elementos geométricos:

In [ ]:
gmsh.model.geo.synchronize()

Ahora definimos algo que nos va a servir para las condiciones de contorno.

Y definimos `Grupos Físicos` para definir bordes, etc

In [ ]:
Empotrado = gmsh.model.addPhysicalGroup(1, [l4])
gmsh.model.setPhysicalName(1,Empotrado,'Empotrado')

In [ ]:
Traccionado = gmsh.model.addPhysicalGroup(1, [l2])
gmsh.model.setPhysicalName(1,Traccionado,'Traccionado')

In [ ]:
Superficie = gmsh.model.addPhysicalGroup(2,[S1])
gmsh.model.setPhysicalName(2,Superficie, 'Superficie')

Finalemente generamos el mallado

In [ ]:
gmsh.model.mesh.generate(2)

# El objeto mesh

El mallado es un objeto del cual podemos recuperar la información necesaria para interactuar con nuestro motor de elementos finitos. Por ejemplo para la informacion de los nodos:

In [ ]:
NodeInfo = gmsh.model.mesh.get_nodes()

In [ ]:
NodeInfo[1].shape

In [ ]:
NodeInfo

esta variable node info guarda:
- los numeros de nodo
- la matriz de nodo

In [ ]:
NumeroNodos = NodeInfo[0].shape[0]

In [ ]:
NumeroNodos

es la cantidad de nodos del modelo.

La matriz de nodos es dada en lista, nos conviene pasarla a matriz para tener lo que venimos trabajando:

In [ ]:
MN = NodeInfo[1].reshape(NumeroNodos , 3)

In [ ]:
MN

## Elementos

Podemos obtener las etiquetas de los triangulos y los nodos que los conforman pidiendo los elementos de tipo 2, que son los triángulos:

In [ ]:
ETAGS, ELEMENTS = gmsh.model.mesh.get_elements_by_type(2)

la cantidad de etiquetas nos va a dar el número de elementos,

In [ ]:
NELEM = ETAGS.shape[0]

In [ ]:
ELEMENTS

la matriz de conectividad también hay que reformatear para tener lo que estamos acostumbrados.

In [ ]:
MC = ELEMENTS.reshape([NELEM,3])

In [ ]:
MC

# Condiciones de contorno

Con las definiciones de los Physical Groups, podemos sacar los índices de los nodos que estan empotrados o traccionados. Necesitamos definir una 'entidad' para los elements Traccionados:

In [ ]:
entityTraccionada = gmsh.model.getEntitiesForPhysicalGroup(1, Traccionado)

In [ ]:
print(entityTraccionada[0])

con esa entidad, podemos definir los elementos tipo línea:

In [ ]:
Tgroup, Ttraccionada, Ltraccionada = gmsh.model.mesh.getElements(1, entityTraccionada[0])

In [ ]:
Ttraccionada

In [ ]:
Ltraccionada = Ltraccionada[0].reshape(Ttraccionada[0].shape[0],2)

In [ ]:
Ltraccionada 

Con los elementos líneas traccionadas podemos calcular las longitudes y distribuir la fuerza externa. 

In [ ]:
Longitudes = np.abs( MN[Ltraccionada[:,0]-1,1] - MN[Ltraccionada[:,1]-1,1] )

Ahora puedo calcular las fuerzas externas

In [ ]:
Fuerzas = np.zeros((2*NumeroNodos,1))
espesor = 1
tension = 1000 #Pa

for l, linea in enumerate(Ltraccionada):
    Flocal = np.array([[1],[1]])*tension*espesor*Longitudes[l]
    n1 = linea[0]
    n2 = linea[1]
    #print(Flocal)
    Fuerzas[ np.array([2*(n1-1), 2*(n2-1)], dtype=int)] += Flocal
    

# Empotrados

Para definir los nodos empotrados necesito solamente los índices de los nodos en dicho physical group

In [ ]:
NodosEmpotrados = gmsh.model.mesh.get_nodes_for_physical_group(1,Empotrado)

In [ ]:
NodosEmpotrados[0]

Con eso ya puedo calcular los valores de los índices de los nodos que resultarán empotrados. Cuidado ! las numeraciones de gmsh empiezan desde 1 !

In [ ]:
s = []
r = np.arange(2*NumeroNodos)
for n, nodo in enumerate(NodosEmpotrados[0]):
    s.append(
        np.linspace(2*nodo, 2*nodo +1, 2)
    )
s = np.array(s).astype(int)

luego saco de r todo lo que puse en s:

In [ ]:
r = np.delete( r, s )

In [ ]:
Fuerzas[r]

# Nos divertimos un rato

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import quiver

In [ ]:
plt.style.use('default')
plt.rc('figure',figsize=(15,10))
plt.rc('font', size=22)

In [ ]:
Fx = Fuerzas[2*np.arange(NumeroNodos)]
Fy = Fuerzas[2*np.arange(NumeroNodos)+1]

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0.1, 0.2, 0.5, 0.6])
ax.triplot(MN[:,0],MN[:,1],MC-MC.min(), )
ax.quiver( MN[:,0], MN[:,1], Fx, Fy, linewidth=5, units='width', scale=1e5)
ax.set_xlim(-1, 2*L+(Fx/1e3).max())
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')

# Agregar Resulatados

Inventemos por ahora un resultado vectorial

In [ ]:
DESP = np.hstack((MN[:,0].reshape(-1,1)*0.1, MN[:,1].reshape(-1,1)*0, MN[:,2].reshape(-1,1)*0))

In [ ]:
plt.plot(MN[:,0], DESP[:,0],'o')
plt.ylabel('$\Delta x (m)$')
plt.xlabel('$x (m)$')

Devemos iniciar una visualizacion:

In [ ]:
desps = gmsh.view.add("desp")

devemos guardar esos datos en la visualizacion:

In [ ]:
Desps = gmsh.view.addModelData(desps, 0, 'test2', 'NodeData', NodeInfo[0], DESP, numComponents=3)

In [ ]:
desps

In [ ]:
!gmsh --version

In [ ]:
gmsh.option.setNumber(f'View[{desps}].VectorType',5)

Guardo las fuerzas externas

In [ ]:
Fxyz = np.hstack((Fuerzas.reshape(NumeroNodos,2), np.zeros((NumeroNodos,1))))

In [ ]:
fx = gmsh.view.add('fuerzas')
Fz = gmsh.view.addModelData(fx, 0, 'test2','NodeData',NodeInfo[0], Fxyz, numComponents=3)

In [ ]:
gmsh.option.setNumber(f'View[{fx}].VectorType',4)

In [ ]:
gmsh.option.setNumber(f'View[{fx}].GlyphLocation',2)

Ahora le agrego un view para tensiones

In [ ]:
v_t = gmsh.view.add("tension")

In [ ]:
T = MC.mean(axis=1).reshape(-1,1)

In [ ]:
Tensiones = gmsh.view.addModelData(v_t, 0, 'test2', 'ElementData', ETAGS, T , numComponents=1 )

vamos a jugar tambien con estos datos escalares, dependientes del tiempo:

In [ ]:
Temps = gmsh.view.add("Temperaturas")

In [ ]:
for i in range(100):
    gmsh.view.addModelData(Temps,i, 'test2','NodeData',NodeInfo[0],MN[:,0].reshape(-1,1)*i,numComponents=1)

In [ ]:
gmsh.fltk.run()

# Guardar lel mesh de entrada (input) sin resultados 

Escribamos estos resultados con el mesh para ver mas tarde. Por razones de legibilidad humana, queremos escribir el mallado en versión 2 y no en version 4 (el default)

In [ ]:
gmsh.option.setNumber("Mesh.MshFileVersion", 2.2)
gmsh.write('TestMeshView_IN.msh')

# Guardar el mesh con resultados

In [ ]:
gmsh.view.write(1,"TestMeshView_OUT.msh",append=True)

visualicemos el resultado:

# Visualizar el estado final

In [ ]:
gmsh.option.setNumber("Mesh.LineWidth", 5)
gmsh.fltk.run()

In [ ]:
gmsh.finalize()